# SQL practice on chinook music service database

We are going to practice SQL techniques learned in the Intermediate SQL Course.
Here is our database schema
![](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg)

Firstly we are going to connect to the database

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

Then observe the result

In [2]:
%%sql

SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


# 1. The most sold genres in the USA

The case is that Chinook company is going to sign a deal with a new record label which specializes in artists from the USA. 
This label have 4 albums, each in different genre and we are offered to choose the first three to be added to the store first.
The genres are:
1. Hip-Hop
2. Punk
3. Pop
4. Blues

We are going to write a query which returns information about the most popular genres sold in the USA.
For this purpose we need to combine different tables:
1. Customers
2. Invoice
3. Invoice_line
4. Track
5. Genre

As we want to see sales where there are no null values in any field, we are going to use INNER JOIN in every conection.

First, we compute the sum of all sales in the USA to bear it in mind for further cross checking

In [3]:
%%sql
SELECT 
    SUM(i.total) AS tot_sum
FROM invoice AS i
INNER JOIN customer AS c ON i.customer_id = c.customer_id
WHERE c.country = 'USA'

 * sqlite:///chinook.db
Done.


tot_sum
1040.4900000000005


Then we are going to create a middle-table (view) which shows us sales in the USA. In case we will decide further to restart our kernel we should first check if the view is already exists and if so - delete it to avoid an error

In [4]:
%%sql
DROP VIEW IF EXISTS usa_sales;

CREATE VIEW usa_sales AS
SELECT
    g.name AS genre_name,
    CAST((SUM(il.unit_price)) AS int) AS total_sold
FROM
    customer AS c
    INNER JOIN invoice AS i
        ON i.customer_id = c.customer_id
    INNER JOIN invoice_line AS il
        ON i.invoice_id = il.invoice_id
    INNER JOIN track AS t
        ON il.track_id = t.track_id
    INNER JOIN genre AS g
        ON t.genre_id = g.genre_id
WHERE country = 'USA'        
GROUP BY genre_name
ORDER BY total_sold DESC;

SELECT * FROM usa_sales
LIMIT 5

 * sqlite:///chinook.db
(sqlite3.OperationalError) disk I/O error
[SQL: DROP VIEW IF EXISTS usa_sales;]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


All we have left to do is to calculare the percentage for each genre and present the results

In [5]:
%%sql
SELECT
    genre_name,
    total_sold,
    CAST(total_sold * 100 / (SELECT SUM(total_sold) FROM usa_sales) AS int) || '%' '%'
FROM usa_sales
GROUP BY genre_name
ORDER BY total_sold DESC

 * sqlite:///chinook.db
Done.


genre_name,total_sold,%
Rock,555,54%
Alternative & Punk,128,12%
Metal,122,11%
R&B/Soul,52,5%
Blues,35,3%
Alternative,34,3%
Pop,21,2%
Latin,21,2%
Hip Hop/Rap,19,1%
Jazz,13,1%


As we can see we have rock-oriented clients in the USA. So we must choose Punk and Blues first. The difference between Pop and Hip-Hop is so tiny that doesnt show any obvious deсision

# 2. Sales agents' performance
Next we are going to analyze the perfomance of each sales manager. We will create a query to see the sum of sales for each of them

In [6]:
%%sql
SELECT 
    e.first_name || " " || e.last_name AS employee_name,
    ROUND((SUM(i.total)),1) AS total_sales
    
FROM employee AS e

    INNER JOIN customer AS c
    ON e.employee_id = c.support_rep_id
    
    INNER JOIN invoice AS i 
    ON c.customer_id = i.customer_id
    
GROUP BY 1

 * sqlite:///chinook.db
Done.


employee_name,total_sales
Jane Peacock,1731.5
Margaret Park,1584.0
Steve Johnson,1393.9


There are obvious difference in the results. But we can't be sure that they all had the same working period. Let's add a hire_date column

In [7]:
%%sql
SELECT 
    e.first_name || " " || e.last_name AS employee_name,
    e.hire_date,
    ROUND((SUM(i.total)),1) AS total_sales
    
FROM employee AS e

    INNER JOIN customer AS c
    ON e.employee_id = c.support_rep_id
    
    INNER JOIN invoice AS i 
    ON c.customer_id = i.customer_id
GROUP BY 1
ORDER BY 3 DESC

 * sqlite:///chinook.db
Done.


employee_name,hire_date,total_sales
Jane Peacock,2017-04-01 00:00:00,1731.5
Margaret Park,2017-05-03 00:00:00,1584.0
Steve Johnson,2017-10-17 00:00:00,1393.9


We can see that the hire date is quite different for all 3 sales managers. It seems interesting to compute monthly average sales for each. 
Before doing this we want to check the date of the last invoice in our database so we can calculate the period of work for each manager

In [8]:
%%sql
SELECT 
    e.first_name || " " || e.last_name AS employee_name,
    e.hire_date,
    i.invoice_date,
    SUM(i.total) AS total_sales
    
FROM employee AS e

    INNER JOIN customer AS c
    ON e.employee_id = c.support_rep_id
    
    INNER JOIN invoice AS i 
    ON c.customer_id = i.customer_id
GROUP BY 1,3
HAVING MAX(i.invoice_date)
ORDER BY 1,3
LIMIT 10

 * sqlite:///chinook.db
Done.


employee_name,hire_date,invoice_date,total_sales
Jane Peacock,2017-04-01 00:00:00,2017-01-03 00:00:00,25.740000000000002
Jane Peacock,2017-04-01 00:00:00,2017-01-06 00:00:00,7.92
Jane Peacock,2017-04-01 00:00:00,2017-01-13 00:00:00,9.9
Jane Peacock,2017-04-01 00:00:00,2017-01-18 00:00:00,8.91
Jane Peacock,2017-04-01 00:00:00,2017-01-20 00:00:00,10.89
Jane Peacock,2017-04-01 00:00:00,2017-01-21 00:00:00,3.96
Jane Peacock,2017-04-01 00:00:00,2017-01-25 00:00:00,3.96
Jane Peacock,2017-04-01 00:00:00,2017-01-26 00:00:00,8.91
Jane Peacock,2017-04-01 00:00:00,2017-02-02 00:00:00,4.95
Jane Peacock,2017-04-01 00:00:00,2017-02-06 00:00:00,9.9


We can see some surprizing data where invoice date is earlier than employee hire date. This could be happened when a customer who made a purchace was assigned to a sales-agent who is not working now but had the same employee_id as one of current agents. This is why all IDs must be unique! Even if an employee resigned.
<br /><br />
But nevertheless we can figure out the date of the last sale for each agent

In [9]:
%%sql
SELECT 
    e.first_name || " " || e.last_name AS employee_name,
    i.invoice_date AS last_invoice_date,
    SUM(i.total) AS total_sales   
FROM employee AS e

    INNER JOIN customer AS c
    ON e.employee_id = c.support_rep_id
    
    INNER JOIN invoice AS i 
    ON c.customer_id = i.customer_id

GROUP BY 1
HAVING MAX(i.invoice_date)
ORDER BY 1,3 DESC

 * sqlite:///chinook.db
Done.


employee_name,last_invoice_date,total_sales
Jane Peacock,2020-12-30 00:00:00,1731.5100000000039
Margaret Park,2020-12-29 00:00:00,1584.0000000000034
Steve Johnson,2020-12-20 00:00:00,1393.920000000002


As we can see for all our agents it is December 2020. Now we can compute sales per one month of work

In [10]:
%%sql
SELECT 
    e.first_name || " " || e.last_name AS employee_name,
    CAST(((JULIANDAY('2020-12-31') - JULIANDAY(hire_date))/365*12) AS int) AS period_months,
    ROUND((SUM(i.total) / ((JULIANDAY('2020-12-31') - JULIANDAY(hire_date))/365*12)), 2) AS sales_per_month
        
FROM employee AS e

    INNER JOIN customer AS c
    ON e.employee_id = c.support_rep_id
    
    INNER JOIN invoice AS i 
    ON c.customer_id = i.customer_id
GROUP BY 1
ORDER BY 3 DESC

 * sqlite:///chinook.db
Done.


employee_name,period_months,sales_per_month
Jane Peacock,45,38.44
Steve Johnson,38,36.21
Margaret Park,43,36.01


As we can see from the table above the performance of Jane Peacock is slightly better than others

# 3. Sales by countries

Our next mission is to create a table, which shows sales information for each country:
1. Total number of customers
2. Total value of sales
3. Average value of sales per customer
4. Average order value

Moreover we were tasked to group all counties with only one customer to 'Others' and sort the result by the total sales from highest to lowest with 'Others' group at the very bottom. Quite tricky a challenge!

We will use the same technique as above and first create a view and then make a query from it

In [11]:
%%sql
DROP VIEW IF EXISTS countries_with_others;

CREATE VIEW countries_with_others AS

SELECT 
    c.country,
    COUNT(DISTINCT c.customer_id) AS num_of_customers,
    ROUND(SUM(i.total),1) AS total_sales,
    ROUND(SUM(i.total) / COUNT(DISTINCT c.customer_id),1) AS sales_per_customer,
    COUNT(DISTINCT i.invoice_id) AS num_of_orders,
    ROUND(SUM(i.total) / COUNT(DISTINCT i.invoice_id),1) AS avg_order_sum,
    CASE
        WHEN COUNT(DISTINCT c.customer_id) < 2 THEN 'Others'
        ELSE c.country
        END AS country_group
FROM 
    customer AS c
    INNER JOIN invoice AS i
        ON c.customer_id = i.customer_id
GROUP BY 1;

SELECT * FROM countries_with_others
LIMIT 5

 * sqlite:///chinook.db
Done.
Done.
Done.


country,num_of_customers,total_sales,sales_per_customer,num_of_orders,avg_order_sum,country_group
Argentina,1,39.6,39.6,5,7.9,Others
Australia,1,81.2,81.2,10,8.1,Others
Austria,1,69.3,69.3,9,7.7,Others
Belgium,1,60.4,60.4,7,8.6,Others
Brazil,5,427.7,85.5,61,7.0,Brazil


Now we have all data we need to complete a task with an additional column which shows us a country name if there are 2 or more customers, and returns 'Others' if there are only one.

We now need to regroup our results by new column 'Group' and then sort it the special way (for 'Others' to be at the bottom)

For the first part we'll create another view - grouped as we wanted, but yet unsorted

In [12]:
%%sql
DROP VIEW IF EXISTS with_others_unsorted;

CREATE VIEW with_others_unsorted AS

SELECT 
    country_group AS country,    
    SUM(num_of_customers) AS num_of_customers,
    SUM(total_sales) AS total_sales,
    ROUND(SUM(total_sales) / SUM(num_of_customers),1) AS sales_per_customer,
    SUM(num_of_orders) AS num_of_orders,
    ROUND((SUM(total_sales) / SUM(num_of_orders)),1) AS avg_order_sum
FROM 
    countries_with_others   
GROUP BY 1
ORDER BY 2 DESC;

SELECT * FROM with_others_unsorted

 * sqlite:///chinook.db
Done.
Done.
Done.


country,num_of_customers,total_sales,sales_per_customer,num_of_orders,avg_order_sum
Others,15,1094.8,73.0,147,7.4
USA,13,1040.5,80.0,131,7.9
Canada,8,535.6,67.0,76,7.0
France,5,389.1,77.8,50,7.8
Brazil,5,427.7,85.5,61,7.0
Germany,4,334.6,83.7,41,8.2
United Kingdom,3,245.5,81.8,28,8.8
Portugal,2,185.1,92.6,29,6.4
India,2,183.2,91.6,21,8.7
Czech Republic,2,273.2,136.6,30,9.1


The last step is to sort the result using the technique produced in the guide to this project

In [13]:
%%sql
SELECT 
    country,    
    num_of_customers,
    total_sales,
    sales_per_customer,   
    avg_order_sum

FROM 
    (SELECT 
        *,
        CASE
            WHEN country = 'Others' THEN 1
            ELSE 0
        END AS sorting_col
        FROM
            with_others_unsorted
        ORDER BY sorting_col
    )

 * sqlite:///chinook.db
Done.


country,num_of_customers,total_sales,sales_per_customer,avg_order_sum
USA,13,1040.5,80.0,7.9
Canada,8,535.6,67.0,7.0
France,5,389.1,77.8,7.8
Brazil,5,427.7,85.5,7.0
Germany,4,334.6,83.7,8.2
United Kingdom,3,245.5,81.8,8.8
Portugal,2,185.1,92.6,6.4
India,2,183.2,91.6,8.7
Czech Republic,2,273.2,136.6,9.1
Others,15,1094.8,73.0,7.4


# 4. Whole albums sales

This time imagine we are considering whether to buy only the most popular tracks from each album or continue to pay record companies some extra for the whole albums.

We are asked to separate all purchases into 2 categories: album purchases and purchases of particular tracks to find out how often people buy whole albums.

We are going to manage the task by several steps:
1. From each invoice we will take any song (doesn't matter which) and determine which album does this song belong to. Then we can connect each invoice with an album. The results will be put into a view.
2. For each invoice we will create a list of included tracks and compare the result with the list of tracks from the album connected with this invoice (mentioned above). If all the tracks from the album are presented in the invoice, we'll consider this purchase as an "album purchase".
3. Then we will count how many album and non-album purchases we had and compute the percentage of each category.

STEP 1:

In [14]:
%%sql
DROP VIEW IF EXISTS invoice_album;

CREATE VIEW invoice_album AS
SELECT 
    il.invoice_id,
    t.album_id

FROM 
    invoice_line AS il   
    INNER JOIN track AS t            
    ON il.track_id = t.track_id
GROUP BY 1    
ORDER BY 1;

SELECT * FROM invoice_album
LIMIT 3

 * sqlite:///chinook.db
Done.
Done.
Done.


invoice_id,album_id
1,91
2,322
3,203


STEP 2:

In [15]:
%%sql
DROP VIEW IF EXISTS invoice_category;
CREATE VIEW invoice_category AS

SELECT
    ia.invoice_id,
    CASE
        WHEN
             (
              SELECT t.track_id 
              FROM   track t
              WHERE  t.album_id = ia.album_id 

              EXCEPT 

              SELECT il.track_id 
              FROM   invoice_line il
              WHERE  il.invoice_id = ia.invoice_id
                 
             ) IS NULL

        THEN "yes"
        ELSE "no"
        END AS "album_purchase"
FROM invoice_album AS ia;


SELECT * FROM invoice_category
LIMIT 3

 * sqlite:///chinook.db
Done.
Done.
Done.


invoice_id,album_purchase
1,yes
2,no
3,no


STEP 3:

In [16]:
%%sql

SELECT
    album_purchase,
    COUNT(invoice_id) num_of_purchases,
    (
    ROUND
        ( 
        CAST(COUNT(invoice_id) AS FLOAT) / 
                (
                 SELECT COUNT(invoice_id) 
                 FROM invoice_category
                )
        ,4 
        )
        * 100) || '%' AS '%'
FROM 
    invoice_category
GROUP BY 1

 * sqlite:///chinook.db
Done.


album_purchase,num_of_purchases,%
no,484,78.83%
yes,130,21.17%


It is not surpisingly that only 22% of all purchases were the whole album ones. However it could be quite risky to change a strategy and sell only the most popular tracks. The absence of some songs from the album may lead customer to change the music service. I would continue to store and sale the whole albums